[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Isotr0py/SakuraLLM-Notebooks/blob/main/Sakura-13B-Galgame-Colab.ipynb)

In [ ]:
#@title 初始化环境
#@markdown 挂载Google网盘
Mount_GDrive = False # @param {type:"boolean"}
if Mount_GDrive:
  from google.colab import drive

  drive.mount('/content/gdrive')
  ROOT_PATH = "/content/gdrive/MyDrive"
else:
  ROOT_PATH = "/content"
!nvidia-smi

In [ ]:
#@title 安装依赖
%cd $ROOT_PATH
!git clone https://github.com/SakuraLLM/Sakura-13B-Galgame.git

%cd Sakura-13B-Galgame
!pip install -q -r requirements.txt
!pip install -q pyngrok

In [ ]:
#@title 翻译EPUB
MODEL = "SakuraLLM/Sakura-13B-LNovel-v0_8-4bit" # @param ["SakuraLLM/Sakura-13B-LNovel-v0_8-3bit", "SakuraLLM/Sakura-13B-LNovel-v0_8-4bit", "SakuraLLM/Sakura-13B-LNovel-v0_8-8bit"]
EPUB_PATH = "novel.epub" # @param {type:"string"}
OUTPUT_FOLDER = "output/" # @param {type:"string"}

%cd $ROOT_PATH/Sakura-13B-Galgame
!python translate_epub.py \
    --model_name_or_path $MODEL \
    --trust_remote_code \
    --model_version 0.8 \
    --use_gptq_model \
    --text_length 512 \
    --data_path $EPUB_PATH \
    --output_folder $OUTPUT_FOLDER

In [ ]:
#@title 翻译文本文件
MODEL = "SakuraLLM/Sakura-13B-LNovel-v0_8-4bit" # @param ["SakuraLLM/Sakura-13B-LNovel-v0_8-3bit", "SakuraLLM/Sakura-13B-LNovel-v0_8-4bit", "SakuraLLM/Sakura-13B-LNovel-v0_8-8bit"]
DATA_PATH = "novel.txt" # @param {type:"string"}
OUTPUT_PATH = "novel_translated.txt" # @param {type:"string"}

%cd $ROOT_PATH/Sakura-13B-Galgame
!python translate_novel.py \
    --model_name_or_path $MODEL \
    --trust_remote_code \
    --model_version 0.8 \
    --use_gptq_model \
    --text_length 512 \
    --data_path $DATA_PATH \
    --output_path $OUTPUT_PATH

In [ ]:
#@title 运行API后端
#@markdown 使用[ngrok](https://ngrok.com/)进行API映射
ngrokToken = ""  # @param {type:"string"}
if ngrokToken:
  from pyngrok import conf, ngrok
  conf.get_default().auth_token = ngrokToken
  conf.get_default().monitor_thread = False
  ssh_tunnels = ngrok.get_tunnels(conf.get_default())
  if len(ssh_tunnels) == 0:
      ssh_tunnel = ngrok.connect(5000)
      print('address：'+ssh_tunnel.public_url)
  else:
      print('address：'+ssh_tunnels[0].public_url)

MODEL = "SakuraLLM/Sakura-13B-LNovel-v0_8-4bit" # @param ["SakuraLLM/Sakura-13B-LNovel-v0_8-3bit", "SakuraLLM/Sakura-13B-LNovel-v0_8-4bit", "SakuraLLM/Sakura-13B-LNovel-v0_8-8bit"]

!python server.py \
  --model_name_or_path $MODEL \
  --use_gptq_model \
  --model_version 0.8 \
  --trust_remote_code \
  --no-auth